In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

from simulation.emit_particles import getDistances, getDirectedSpread
from simulation.loader import load_environment
from simulation.particle_spread import particle_spread
from simulation.visibility import getVisibilityMaps
from simulation.visualizations import plot_path

In [ ]:
resolution = 3
input_map_path = Path("../data/chalmers_karresturang_normal.txt")


environment = load_environment(str(input_map_path), resolution)
width, height = environment.shape
plot_path(environment)

In [ ]:
visibility_map_path = input_map_path.with_suffix(".visibility.npy")
if visibility_map_path.exists():
    visibilities = np.load(visibility_map_path, allow_pickle=True)
else:
    visibilities = getVisibilityMaps(environment, resolution, screen=False)
    np.save(visibility_map_path, visibilities)

distances = getDistances(visibilities, environment, resolution)
cones = getDirectedSpread(visibilities, environment, resolution)

In [ ]:
density = np.zeros((width, height))

def infection_step(particle_density, infected, emission_rate=3):
    return particle_spread(resolution=resolution, environment=environment, visibilityMatrix=visibilities,
                           particleMatrix=particle_density, distanceMatrix=distances, emissionRate=emission_rate)\
        .emit(infected, None, cones)

plot_path(environment)

density = infection_step(density, [(50, 40)])
plt.imshow(density.T, cmap="Reds")
plt.show()

plt.plot(density[30:70, 40])
plt.show()

x = 1.0 - distances / 2
x[x <= 0] = 0

fig, ax = plt.subplots()
ax.axis("off")
ax.imshow(x, cmap="Reds")
plt.show()

plt.plot(x[:, 12])
plt.show()

plt.imshow(visibilities[100, 4].T, cmap="Blues_r")
plt.show()

# Modelling of risk density

Our goal is to have a realistic, yet simple enough model of the risk related to the spreading of diseases.

So first, let's work out the main factors.

---

While speaking, sneezing or coughing, we produce tiny aerosol particles which can carry the disease.

In this picture you can see the particle behaviour over time and space.

The area directly next to the person is the most dangerous.

Over time the particles drop to the ground and stop being a risk.

---

We use a simple function to consider the distance.

---

In our simulation, a portion of the particles drop the ground each time step.

The risk decreases over time.

---

To also consider walls and shields which stop particles, we use a cone of sight.

---

An area around a person is too simple as we are normally directed in some way.

We use a cone to cover this effect.

---



There are many ways to model the spreading of a disease.

-- picture of research papers

For example, individual particles can be simulated. (They interact with the airflow and move around. One can even
differentiate between different sizes which affect how long they stay in the air.)

In order to have a simple model which still captures the main attributes, we assume that distance is the main risk factor.

People may breath, cough or touch things in irregular intervals and we use the risk as an simplification of all that.

...

The most dangerous place is directly next to an infected agent. The more distance, the less risk is associated.

We use 2 meters as the maximum distance that a infected agent can affect others.

...

Spread is not going to happen through a wall. Therefore, we check for walls.

During sitting, the agent is most likely speaking forwards or sideways. Here we use cones.

...



Next up, Mathias is going to talk about the results.

In [ ]:
def plot_agent_walk():
    fig, ax = plt.subplots(figsize=(6, 6))
    # walk = [(105, y) for y in range(80, 5-1, -1)] + [(x, 5) for x in range(105, 90, -1)]
    walk = [(105, y) for y in range(70, 76)] + [(x, 76) for x in range(105, 95 - 1, -1)] + [(95, y) for y in range(76, 70, -1)]
    m = ax.imshow(visibilities[walk[0]][1:,1:], cmap="Blues_r")
    e = ax.imshow((environment[walk[0][0]-11:walk[0][0]+12, walk[0][1]-11:walk[0][1]+12].T == 0), cmap="Reds", alpha=0.5)

    ax.axis("off")

    ax.add_patch(plt.Circle((11.5, 11.5), resolution / 4, fc='k', ec='k', fill=True))
    # ax.plot([12], [12], "kx")

    fig.tight_layout()

    def update(frame):
        pos = walk[frame]
        x, y = pos
        m.set_data(visibilities[walk[frame]].T[1:,1:])
        e.set_data((environment[x-11:x+12, y-11:y+12].T == 0))
        return m,

    animation = FuncAnimation(fig, update, frames=len(walk))
    animation.save("walk.mp4", fps=6, dpi=300)
    plt.savefig("walk.png")

plot_agent_walk()

## Diffusion

Build up around an agent, until equilirium is reached.

Risk diffuses by multiplying with a

In [ ]:
def plot_diffusion():
    density = np.zeros_like(environment)

    densities_at_point = []

    pos = (50, 40)

    for _ in range(400):
        density = infection_step(density, [pos], emission_rate=1.0)

    for _ in range(50):
        density = infection_step(density, [pos], emission_rate=1.0)
        densities_at_point.append(density[pos])

    for i in range(300):
        density = infection_step(density, [], emission_rate=1.0)
        densities_at_point.append(density[pos])

    fig, ax = plt.subplots(figsize=(5, 3))
    ax.set_xlim((-50 * 2/3, 300* 2/3)); ax.set_ylim((0, 80))
    ax.set_yticks([])
    ax.set_xlabel("time [s]"); ax.set_ylabel("risk density")
    ax.plot(np.arange(-50, 300) * 2 / 3, np.array(densities_at_point), "k-")
    ax.axvline(0, ls="--", c="grey")
    fig.tight_layout()
    plt.savefig("risk-diffusion.png", dpi=300)
    plt.show()

plot_diffusion()

In [ ]:
def risk_distance():
    fig, ax = plt.subplots(figsize=(7, 3))
    ax.set_xlim((-0.5, 5.5)) # ax.set_ylim((0, 1.5))
    ax.set_yticks([])
    ax.set_xlabel("distance from infected person [m]"); ax.set_ylabel("risk density")
    ax.set_aspect("equal")
    x = np.linspace(0, 5.5)
    risk = 1 - x/4
    risk[risk <= 0] = 0
    ax.plot(x, 2 * risk, "k-")
    fig.tight_layout()
    plt.savefig("risk-distance.png", dpi=300)
    plt.show()

risk_distance()

In [ ]:
v = np.zeros(10)
v[5] = 1.0

values = []
for _ in range(20):
    v_next = np.zeros_like(v)
    v_next += 0.5 * v
    v_next[1:] += 0.25 * v[:-1]
    v_next[:-1] += 0.25 * v[1:]
    v = v_next
    values.append(v[5])

plt.plot(values)
plt.show()


In [ ]:
def plot_cone():
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.axis("off")
    ax.imshow(cones[0,1][:,1:-1], cmap="Reds")
    fig.tight_layout()
    plt.savefig("cone.png", dpi=300)
    plt.show()

plot_cone()